In [1]:
import pandas as pd
import numpy as np
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 먼저 역 위,경도데이터
station = pd.read_csv('data/station_coordinate.csv')
station.to_excel('data/station_total.xlsx')

In [3]:
# 결측치는 직접 찾아 넣었음
station = pd.read_excel('data/station_total.xlsx')
station

,Unnamed: 0,line,name,code,lat,lng
0,0,01호선,녹양,1908.0,37.759380,127.042292
1,1,01호선,남영,1002.0,37.541021,126.971300
2,2,01호선,용산,1003.0,37.529849,126.964561
3,3,01호선,노량진,1004.0,37.514219,126.942454
4,4,01호선,대방,1005.0,37.513342,126.926382
...,...,...,...,...,...,...
725,725,인천선,지식정보단지,3135.0,37.378384,126.645168
726,726,인천선,인천대입구,3136.0,37.386007,126.639484
727,727,인천선,센트럴파크,3137.0,37.393054,126.634729
728,728,인천선,국제업무지구,3138.0,37.399907,126.630347


In [4]:
# 아파트 위경도데이터(ID있음)
where = pd.read_excel('data/03_결측치처리후_주소지_4천개.xlsx')
where.drop(['Unnamed: 0'], axis=1, inplace=True)

# 이하 데이터 전처리
base = pd.read_excel('data/ID있는버전.xlsx')
base['면적당매매가'] = base['거래금액(만원)'] / base['전용면적(㎡)']
base['저층여부'] = np.where(base['층']<5, '저층', '고층')
base['계약년월'] = pd.to_datetime(base['계약년월'], format='%Y%m')
base['분기'], base['년도'] = (base['계약년월'].dt.quarter, base['계약년월'].dt.year)
base['분기'] = base[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del base['년도']
del base['계약일']
base.drop(base[base['전용면적(㎡)']<=10].index, inplace=True)

# 거래금액, 전용면적, 건축년도 모두 사용해 보기로 함
base['계약년월'] = pd.to_datetime(base['계약년월'], format='%Y%m')
base['분기'], base['년도'] = (base['계약년월'].dt.quarter, base['계약년월'].dt.year)
base['분기'] = base[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del base['년도']
table1 = pd.pivot_table(base, values=['거래금액(만원)'], index=['분기'], columns=['ID'], aggfunc=np.mean)
table2 = pd.pivot_table(base, values=['전용면적(㎡)'], index=['분기'], columns=['ID'], aggfunc=np.mean)
table3 = pd.pivot_table(base, values=['건축년도'], index=['분기'], columns=['ID'], aggfunc=np.mean)
check_null1 = table1.isna().sum() / 18
check_null2 = table2.isna().sum() / 18
check_null3 = table3.isna().sum() / 18
check_null1[check_null1 >= 0.5]
check_null2[check_null2 >= 0.5]
check_null3[check_null3 >= 0.5]
remove_cols1 = check_null1[check_null1 >= 0.5].keys()
remove_cols2 = check_null2[check_null2 >= 0.5].keys()
remove_cols3 = check_null3[check_null3 >= 0.5].keys()
table1 = table1.drop(remove_cols1, axis=1)
table2 = table2.drop(remove_cols2, axis=1)
table3 = table3.drop(remove_cols3, axis=1)
table1.fillna(method='ffill', inplace=True)
table1.fillna(method='bfill', inplace=True)
table2.fillna(method='ffill', inplace=True)
table2.fillna(method='bfill', inplace=True)
table3.fillna(method='ffill', inplace=True)
table3.fillna(method='bfill', inplace=True)
data1 = table1.transpose()
data2 = table2.transpose()
data3 = table3.transpose()
money = data1.droplevel(axis=0, level=0)
width = data2.droplevel(axis=0, level=0)
year = data3.droplevel(axis=0, level=0)
data = pd.concat([money,width,year], axis=1)
data = data.round(1)

In [5]:
# 데이터 결합
dic = {}
number = 0

for i1 in range(1,18):
    dic[f'data{i1}']=data.iloc[:,[number, number+18, number+36]]
    number += 1

In [6]:
# 확인
dic['data1']

분기,2018/1,2018/1,2018/1
ID,,,
서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,80.0,1987.0
서울특별시 강남구 개포동 개포주공1단지 40,145736.8,40.0,1982.0
서울특별시 강남구 개포동 개포주공5단지 60,136666.7,60.0,1983.0
서울특별시 강남구 개포동 개포주공5단지 80,177416.7,80.0,1983.0
서울특별시 강남구 개포동 개포주공6단지 50,125900.0,50.0,1983.0
...,...,...,...
서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,60.0,1998.0
서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,80.0,2005.0
서울특별시 중랑구 중화동 한신아파트 50,31756.2,50.0,1997.0


In [7]:
# 출발지점과 도착지점 저장
stay_df = pd.DataFrame({'위도':station['lat'].values,'경도':station['lng'].values})
address_df = pd.DataFrame({'위도':where['위도'].values,'경도':where['경도'].values})
stay_df, address_df

(            위도          경도
 0    37.759380  127.042292
 1    37.541021  126.971300
 2    37.529849  126.964561
 3    37.514219  126.942454
 4    37.513342  126.926382
 ..         ...         ...
 725  37.378384  126.645168
 726  37.386007  126.639484
 727  37.393054  126.634729
 728  37.399907  126.630347
 729  37.467048  126.707938
 
 [730 rows x 2 columns],
              위도          경도
 0     37.483372  127.053437
 1     37.483381  127.057068
 2     37.487868  127.068988
 3     37.487868  127.068988
 4     37.488481  127.072502
 ...         ...         ...
 4089  37.605430  127.082936
 4090  37.599447  127.078938
 4091  37.595961  127.080314
 4092  37.595961  127.080314
 4093  37.595961  127.080314
 
 [4094 rows x 2 columns])

In [8]:
# 거리계산
g = []

for i in range(len(stay_df)):
    st_name = station['name'][i]
    st_line = station['line'][i]
    start = (float(stay_df['위도'][i]), float(stay_df['경도'][i]))
    for j in range(len(address_df)):
        apt_name = where['ID'][j]
        goal = (float(address_df['위도'][j]), float(address_df['경도'][j]))
        geoly = haversine(start, goal)
        r_start = np.round(start, 2)
        r_goal = np.round(goal, 2)
        r_geoly = np.round(geoly, 2)
        if r_geoly <=0.5:
            g.append([st_name, st_line, r_start, apt_name, r_goal, r_geoly])
        else:
            pass

In [9]:
# 데이터프레임화
g_df = pd.DataFrame(g, columns=['역 이름', '역 호선', '역 좌표', '아파트 이름', '아파트 좌표', '역까지의 거리'])
g_df.head()

,역 이름,역 호선,역 좌표,아파트 이름,아파트 좌표,역까지의 거리
0,남영,01호선,"[37.54, 126.97]",서울특별시 용산구 문배동 용산KCC웰츠타워 80,"[37.54, 126.97]",0.49
1,남영,01호선,"[37.54, 126.97]",서울특별시 용산구 문배동 이안용산1차 80,"[37.54, 126.97]",0.49
2,남영,01호선,"[37.54, 126.97]",서울특별시 용산구 원효로1가 용산더프라임 80,"[37.54, 126.97]",0.22
3,용산,01호선,"[37.53, 126.96]",서울특별시 용산구 한강로2가 벽산메가트리움 80,"[37.53, 126.97]",0.45
4,용산,01호선,"[37.53, 126.96]",서울특별시 용산구 한강로2가 한강로대우아이빌 30,"[37.53, 126.97]",0.45


In [10]:
# 검색 실증
def search(x):
    x = g_df[g_df['아파트 이름']==x]
    return x
len(search('미국'))

0

In [74]:
# 실험1
base['ID'].iloc[0]

'서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80'

In [72]:
# 실험2
dic['data1'].iloc[1]

분기
2018/1    145736.8
2018/1        40.0
2018/1      1982.0
Name: 서울특별시 강남구 개포동 개포주공1단지 40, dtype: float64

In [77]:
# 인접역 산출
p1 = []
n = 0
for i in range(len(base['ID'])):
    a = base['ID'].iloc[n]
    b = len(search(base['ID'].iloc[n]))
    p1.append([a,b])
    n+=1
# test = pd.concat([dic['data1'], p1], axis=1)
p1_df = pd.DataFrame(p1, columns=['ID','인접역 수'])

In [78]:
# 인접역 산출 결과
p1_df.head()

,ID,인접역 수
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,0
1,서울특별시 동작구 신대방동 롯데관악타워 150,0
2,서울특별시 동작구 신대방동 경남교수 80,0
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,1
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,1


In [83]:
# 길이
len(p1_df)

281197

In [102]:
# 이하부터는 주석처리 때문에 처음 작동시 에러날 수 있음
# 실험 중 발생된 무의미한 코드
# p1_df.set_index('ID', drop=True, inplace=True)
p1_df.reset_index(inplace=True)
p1_df.head()

,ID,인접역 수
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,0
1,서울특별시 동작구 신대방동 롯데관악타워 150,0
2,서울특별시 동작구 신대방동 경남교수 80,0
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,1
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,1


In [115]:
# 기존 거래금액 등의 피쳐값을 보기좋게 정리 및 멀티인덱스 제거
test2 = pd.DataFrame(dic['data1'])
# test2.reset_index(inplace=True)
test2.drop(labels='index', axis=1, inplace=True)
test2.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
test2

,ID,거래금액(만원),전용면적,건축년도
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,80.0,1987.0
1,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,40.0,1982.0
2,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,60.0,1983.0
3,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,80.0,1983.0
4,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,50.0,1983.0
...,...,...,...,...
4089,서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,60.0,1998.0
4090,서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,80.0,2005.0
4091,서울특별시 중랑구 중화동 한신아파트 50,31756.2,50.0,1997.0
4092,서울특별시 중랑구 중화동 한신아파트 60,36534.6,60.0,1997.1


In [116]:
# 인접역 데이터와 결합(2018년 1분기 데이터)
test1 = pd.merge(test2, p1_df)
test1.drop_duplicates()

,ID,거래금액(만원),전용면적,건축년도,인접역 수
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,80.0,1987.0,0
24,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,40.0,1982.0,1
146,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,60.0,1983.0,1
171,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,80.0,1983.0,1
199,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,50.0,1983.0,1
...,...,...,...,...,...
214320,서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,60.0,1998.0,1
214339,서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,80.0,2005.0,1
214360,서울특별시 중랑구 중화동 한신아파트 50,31756.2,50.0,1997.0,5
214443,서울특별시 중랑구 중화동 한신아파트 60,36534.6,60.0,1997.1,5
